In [3]:
#library import
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
import os, random, json
spark=SparkSession.builder.master("local").appName("studentmarkpractice").getOrCreate()

In [10]:
class Prelanding_to_landing:
    
    def __init__(self,file_parameter):
        
        """function to get input parameters to read the file from local path"""
        self.file_parameter=file_parameter
        
    def read_csv(self):
        return spark.read.format(self.file_parameter['src_format']).option("header","True").load(self.file_parameter['src_path'])
    
    def write_parquet(self,df):
        df.write.format(self.file_parameter['dest_format']).mode("overwrite").option("path",self.file_parameter['dest_path']).saveAsTable("school.{0}".format(self.file_parameter['view_name']))
    
    def open_dir(self):
        os.chdir(self.file_parameter['dest_path'])
        dest_file_path=f"{self.file_parameter['view_name']}.parquet"
        for file in os.listdir():
        # Check whether file is in text format or not
            if file.endswith(".parquet"):
                src_file_path = f"{file}"
                os.rename(src_file_path,dest_file_path)
            else:
                src_file_path = f"{file}"
                os.remove(src_file_path)
    
    def main_block(self):
        
        """main execution block"""
        df=self.read_csv()
        self.write_parquet(df)
        self.open_dir()
        

In [11]:
if __name__=='__main__':
    file_parameter1={
        "src_format":"csv",
        "src_path":"C:/Git_Repo/big_data_project/data_files/pre_landing/raw_files/school/student/studentdetails.csv",
        "dest_format":"parquet",
        "dest_path":"C:/Git_Repo/big_data_project/data_files/landing/school_landing/student",
        "view_name":"studentdetails"
    }
    
    file_parameter2={
        "src_format":"csv",
        "src_path":"C:/Git_Repo/big_data_project/data_files/pre_landing/raw_files/school/subject/subjectdetails.csv",
        "dest_format":"parquet",
        "dest_path":"C:/Git_Repo/big_data_project/data_files/landing/school_landing/subject",
        "view_name":"subjectdetails"
    }
    
    file_parameter3={
        "src_format":"csv",
        "src_path":"C:/Git_Repo/big_data_project/data_files/pre_landing/raw_files/school/marks/markdetails.csv",
        "dest_format":"parquet",
        "dest_path":"C:/Git_Repo/big_data_project/data_files/landing/school_landing/marks",
        "view_name":"markdetails"
    }
    
    spark.sql("create database if not exists {0}".format("school"))
    landing_obj1=Prelanding_to_landing(file_parameter1)
    landing_obj2=Prelanding_to_landing(file_parameter2)
    landing_obj3=Prelanding_to_landing(file_parameter3)
    landing_obj1.main_block()
    landing_obj2.main_block()
    landing_obj3.main_block()
    